In [103]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  LSTM , BatchNormalization


In [104]:
import yfinance as yf

In [105]:
period = '1000d'

#tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'TSLA', 'NVDA', 'PYPL', 'ADBE','BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD','BCH-USD', 'BNB-USD', 'LINK-USD', 'ADA-USD', 'XLM-USD', 'SOL-USD', 'TRX-USD']

tickers = ['SOL-USD']


In [106]:
def indir_ve_df_olustur(tickers, period):
    # Her bir hisse senedi için boş bir sözlük oluşturun
    pariteler = {}

    # Her bir hisse senedi için döngü oluşturun ve verileri indirin
    for ticker in tickers:
        try:
            # Hisse senedi verilerini indirin
            veri = yf.download(ticker, period=period)
            
            # Veriyi sözlüğe ekleyin
            pariteler[ticker] = veri
        except Exception as e:
            print(f"{ticker} için veri indirilirken bir hata oluştu: {str(e)}")

    # Her bir hisse senedi için ayrı bir veri çerçevesi oluşturun
    df_listesi = [veri for veri in pariteler.values()]

    return df_listesi

In [107]:
#pariteler adında klasör oluştur :

import os 

if not os.path.exists('pariteler'):
    os.makedirs('pariteler')



In [108]:
datasets = indir_ve_df_olustur(tickers, period)

#pariteler içine kaydet 

for i in range(len(datasets)):
    datasets[i].to_csv(f'pariteler/{tickers[i]}.csv')
    #print(f'{tickers[i]} verisi csv olarak kaydedildi')


[*********************100%%**********************]  1 of 1 completed


In [109]:
window = 30

def veri_hazirla(df, window):
    # Veri çerçevesinin kopyasını oluşturun
    df = df.copy()
    
    # Veri çerçevesine yeni sütunlar ekleyin
    for i in range(1, window + 1):
        df[f'Önceki_{i}_Açılış'] = df['Open'].shift(i)
        df[f'Önceki_{i}_Yüksek'] = df['High'].shift(i)
        df[f'Önceki_{i}_Düşük'] = df['Low'].shift(i)
        df[f'Önceki_{i}_Kapanış'] = df['Close'].shift(i)
        df[f'Önceki_{i}_Adj'] = df['Adj Close'].shift(i)
        df[f'Önceki_{i}_Hacim'] = df['Volume'].shift(i)
        
    # NaN değerleri bırakın
    df.dropna(inplace=True)
    
    return df

# Her bir veri çerçevesi için döngü oluşturun ve verileri hazırlayın
data_windowed = [veri_hazirla(df, window) for df in datasets]


if not os.path.exists('windowed'):
    os.makedirs('windowed')

# Her bir veri çerçevesi için parite ismiyle birlikte windowed klasörüne kaydedin 
for i in range(len(data_windowed)):
    data_windowed[i].to_csv(f'windowed/{tickers[i]}_windowed.csv')
    print(f'{tickers[i]} verisi windowed olarak kaydedildi')



/var/folders/w9/3z_zbj956tj44glgzg44rgbr0000gn/T/ipykernel_42600/2059070395.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Önceki_{i}_Düşük'] = df['Low'].shift(i)
/var/folders/w9/3z_zbj956tj44glgzg44rgbr0000gn/T/ipykernel_42600/2059070395.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Önceki_{i}_Kapanış'] = df['Close'].shift(i)
/var/folders/w9/3z_zbj956tj44glgzg44rgbr0000gn/T/ipykernel_42600/2059070395.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

SOL-USD verisi windowed olarak kaydedildi


In [110]:
#data_windowed

In [111]:
#her bir veri çerçevesi için X  = features(öncekiler) y = labels olan veriyapısını kuruyoruz (labels  = ['Open','High', 'Low', 'Close', 'Adj Close', 'Volume'])

def X_y_olustur(df, window, labels):
    # Veri çerçevesinin kopyasını oluşturun
    df = df.copy()
    
    # Özellikler ve etiketler için boş listeler oluşturun
    X = []
    y = []
    
    # Her bir satır için döngü oluşturun
    for i in range(len(df) - window):
        # Özellikler için satırı alın
        X_row = df.iloc[i:i + window].values
        
        # Etiketler için satırı alın
        y_row = df[labels].iloc[i + window].values
        
        # Özellikleri ve etiketleri listelere ekleyin
        X.append(X_row)
        y.append(y_row)
        
    # Listeleri numpy dizilerine dönüştürün
    X = np.array(X)
    y = np.array(y)
    
    return X, y

# Her bir veri çerçevesi için döngü oluşturun ve X ve y oluşturun
X_y = [X_y_olustur(df, window, ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']) for df in data_windowed]
#X_y = [X_y_olustur(df, window, ['Close' ,'Volume']) for df in data_windowed]

# Her bir veri çerçevesi için parite ismiyle birlikte numpy klasörüne kaydedin

if not os.path.exists('numpy'):
    os.makedirs('numpy')
for i in range(len(X_y)):
    np.save(f'numpy/{tickers[i]}_X.npy', X_y[i][0])
    np.save(f'numpy/{tickers[i]}_y.npy', X_y[i][1])
    print(f'{tickers[i]} verisi X ve y olarak kaydedildi')
    



SOL-USD verisi X ve y olarak kaydedildi


In [112]:
# her bir veri çerçevesi için train test split yapalım time series için doğru yolla yapmak gerekiyor

def train_test_split(X, y, test_size):
        # Test boyutunu hesaplayın
        test_size = int(len(X) * test_size)
        
        # Eğitim ve test veri kümelerini ayırın
        X_train = X[:-test_size]
        X_test = X[-test_size:]
        y_train = y[:-test_size]
        y_test = y[-test_size:]
        
        return X_train, X_test, y_train, y_test

# Her bir veri çerçevesi için döngü oluşturun ve train ve test veri kümelerini ayırın

X_train_test = [train_test_split(X_y[i][0], X_y[i][1], 0.2) for i in range(len(X_y))]

In [113]:
# her bir veri çerçevesi için model oluşturup eğitelim

def model_olustur(input_shape, output_shape):
    # Modeli oluşturun
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(128))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(output_shape))
    
    return model

In [114]:
# her bir veri çerçevesi için modeli eğitelim


if not os.path.exists('models'):
    os.makedirs('models')

epochs = 10
batch_size = 64
models = []

for i in range(len(X_train_test)):
    # Modeli oluşturun
    model = model_olustur(X_train_test[i][0].shape[1:], X_train_test[i][2].shape[1])
    
    # Modeli derleyin
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Modeli eğitin
    model.fit(X_train_test[i][0], X_train_test[i][2], epochs=epochs, batch_size=batch_size,
              validation_data=(X_train_test[i][1], X_train_test[i][3]),
                verbose=1)
    
    # Modeli kaydedin
    model.save(f'models/{tickers[i]}_model.h5')
    
    # Modeli listeye ekleyin
    models.append(model)
    print(f'{tickers[i]} modeli eğitildi ve kaydedildi')


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/mean_squared_error/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/mustafakapici/anaconda3/envs/all/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/mustafakapici/anaconda3/envs/all/lib/python3.11/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/Users/mustafakapici/anaconda3/envs/all/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/Users/mustafakapici/anaconda3/envs/all/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/mustafakapici/anaconda3/envs/all/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Users/mustafakapici/anaconda3/envs/all/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

In [ ]:
# her bir veri çerçevesi için modeli yükleyip tahmin yapalım tahminlerin bütün kolonlarını görselleştirelim

# Her bir model için döngü oluşturun
for i in range(len(models)):
    # Modeli yükleyin
    model = tf.keras.models.load_model(f'models/{tickers[i]}_model.h5')
    
    # Tahminleri yapın
    predictions = model.predict(X_train_test[i][1])
    
    # Tahminleri ve gerçek değerleri görselleştirin
    plt.figure(figsize=(12, 6))
    plt.plot(predictions, label='Tahminler')
    plt.plot(X_train_test[i][3], label='Gerçek Değerler')
    plt.legend()
    plt.show()
    
    